In [2]:
import numpy as np

In [20]:
def FAST_CARATHEODORY_SET(P,P_index,u,k):
    # P n*d array
    # u n array, weight vector for P
    d = P.shape[1]
    # 1.remove all points with zero weight
    weight_zero_index = u == 0
    P = P[weight_zero_index]
    P_index = P_index(weight_zero_index)
    # 2. P is already small
    if(P.shape[0] <= d+1):
        return P,P_index,u
    # 4. partition of P into k subsets
    k_clusters_lable = np.array([np.random.randint(0,k) for i in range(0, P.shape[0])])
    # 5.the weighted mean of P_k
    cls_w_means = np.array([],dtype = np.float64).reshape(0,d)
    cls_ws = np.array([],dtype = np.float64).reshape(0,1)
    for i in range(0,k):
        temp = k_clusters_lable == i
        P_i = P[temp]
        u_i = u[temp]
        denominator = np.sum(u_i)
        numerator =  np.dot(P_i.transpose(),u_i)
        cls_w_means = np.append(cls_w_means,numerator/denominator)
        cls_ws = np.append(cls_ws,denominator)
    # 8. algorithm 16
    selected_means, selected_ws = CARATHEODORY(cls_w_means,cls_ws) 
    selected_index = selected_ws[selected_ws > 0]
    # 9. the union over all selected clusters
    C = np.array([],dtype = np.float64).reshape(0,d)
    C_index = np.array([],dtype = np.int32).reshape(0,1)
    w = np.array([],dtype = np.float64).reshape(0,1)
    for i in selected_index:
        temp = k_clusters_lable == i
        P_i = P[temp]
        index_i = P_index[temp]
        u_i = u[temp]
        w_i = cls_ws[i]*u_i/np.sum(u_i)
        C = np.vstack((C,P_i))
        C_index = np.append(C_index,index_i)
        w = np.append(w,w_i)
    # 12. recursive call
    C,C_index,w = FAST_CARATHEODORY_SET(C,C_index,w,k)
    return(C,C_index,w)

In [31]:
def CARATHEODORY_MATRIX(A,k):
    n,d = A.shape
    P = np.array([],dtype = np.float64).reshape(0,d**2)
    for i in range(0,n):
        a_i = A[i].reshape(-1,1)
        temp = np.dot(a_i,a_i.transpose()).reshape(1,-1)
        P = np.vstack((P,temp))
    P_index = np.arange(0,n)
    u = np.ones(n)/n
    C,C_index,w = FAST_CARATHEODORY_SET(P,P_index,u,k)
    S = np.array([],dtype = np.float64).reshape(0,d)
    for i in range(0,C.shape[0]):
        S_i = np.sqrt(n*w[i])*A[C_index[i]]
        S = np.vstack((S,S_i))
    return S

In [32]:
# test case for FAST_CARATHEODORY_SET
P = np.arange(12).reshape(6,2)
P_index = np.arange(6)
u = np.ones(6)/6
k = 2
FAST_CARATHEODORY_SET(P,P_index,u,k)

TypeError: 'numpy.ndarray' object is not callable

In [33]:
# test case for CARATHEODORY_MATRIX
A = np.arange(12).reshape(6,2)
k = 2
CARATHEODORY_MATRIX(A,k)

TypeError: 'numpy.ndarray' object is not callable